
### Функция(create_maze) - создает лабиринт с использованием рекурсивного случайного прохода.<hr>
> **Входящие параметры: <br>**

- map_height (int): высота лабиринта.
- map_width (int): ширина лабиринта.
- start (tuple[int, int]): координаты начальной точки (по умолчанию случайно на левой стенке).
- end (tuple[int, int]): координаты конечной точки (по умолчанию случайно на правой стенке).

> **Returns:** <br>

- list[list[str]]: лабиринт, представленный в виде списка списков, где каждый элемент - символ.


In [ ]:
#Способ 1: Рекурсивное построение лабиринта с помощью случайного прохода

import random

def create_maze(
    height: int,
    width: int,
    start: tuple[int, int] = None,
    end: tuple[int, int] = None) -> list[list[str]]:
    if height < 3 or width < 3:
        raise ValueError("Размеры лабиринта должны быть не менее 3x3.")

    maze = [['█' for _ in range(width)] for _ in range(height)]

    if not start:
        start = (random.randint(1, height - 2), 0)
    if not end:
        end = (random.randint(1, height - 2), width - 1)

    maze[start[0]][start[1]] = '~'
    maze[end[0]][end[1]] = '$'

    def carve_maze(x, y):
        maze[x][y] = ' '
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
        random.shuffle(directions)
        for dx, dy in directions:
            nx, ny = x + 2 * dx, y + 2 * dy
            if (
                0 <= nx < height
                and 0 <= ny < width
                and maze[nx][ny] == '█'
            ):
                maze[x + dx][y + dy] = ' '
                carve_maze(nx, ny)

    carve_maze(start[0] + 1, start[1])
    return maze

# Пример использования
print("Лабиринт 10x20 со случайными начальной и конечной точками:")
maze = create_maze(10, 20)
for row in maze:
    print(''.join(row))

Лабиринт 10x20 со случайными начальной и конечной точками:
     █       █   █ █
██ █ █ █████ █ █ █ █
   █   █   █   █   █
 ███████ █████████ $
   █ █         █   █
 █ █ █ ███ █████ ███
 █ █ █ █ █     █   █
~█ █ █ █ █████ ███ █
   █         █     █
████████████████████


In [ ]:
#Способ 2: Построение лабиринта с помощью алгоритма Прима
import random

def create_maze(
    height: int,
    width: int,
    start: tuple[int, int] = None,
    end: tuple[int, int] = None) -> list[list[str]]:
    if height < 3 or width < 3:
        raise ValueError("Размеры лабиринта должны быть не менее 3x3.")

    maze = [['█' for _ in range(width)] for _ in range(height)]

    if not start:
        start = (random.randint(1, height - 2), 0)
    if not end:
        end = (random.randint(1, height - 2), width - 1)

    maze[start[0]][start[1]] = '~'
    maze[end[0]][end[1]] = '$'

    def join_cells(x, y, nx, ny):
        maze[nx][ny] = ' '
        maze[x + (nx - x) // 2][y + (ny - y) // 2] = ' '

    frontier = [(1, 1)]
    while frontier:
        x, y = random.choice(frontier)
        frontier.remove((x, y))
        directions = [(0, 2), (2, 0), (-2, 0), (0, -2)]
        random.shuffle(directions)
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if (
                0 <= nx < height - 1
                and 0 <= ny < width - 1
                and maze[nx][ny] == '█'
            ):
                join_cells(x, y, nx, ny)
                frontier.append((nx, ny))

    return maze

# Пример использования
print("Лабиринт 10x20 со случайными начальной и конечной точками:")
maze = create_maze(10, 20)
for row in maze:
    print(''.join(row))

Лабиринт 10x20 со случайными начальной и конечной точками:
████████████████████
█     █     █ █   ██
~ █ ███ █████ █ ████
█ █               █$
█ █ ███ █ ███ ██████
█ █   █ █   █     ██
███ █████████████ ██
█               █ ██
████████████████████
████████████████████


In [ ]:
# Способ 3: Построение лабиринта с помощью алгоритма удаления случайных стен
import random

def create_maze(
    height: int,
    width: int,
    start: tuple[int, int] = None,
    end: tuple[int, int] = None) -> list[list[str]]:
    """
    Функция создает лабиринт с помощью алгоритма удаления случайных стен.

    Args:
        height (int): Высота лабиринта.
        width (int): Ширина лабиринта.
        start (Optional[Tuple[int, int]]): Координаты начальной точки (по умолчанию случайно на левой стенке).
        end (Optional[Tuple[int, int]]): Координаты конечной точки (по умолчанию случайно на правой стенке).

    Returns:
        list[list[str]]: Лабиринт, представленный в виде списка списков, где каждый элемент - символ.
    """
    if height < 3 or width < 3:
        raise ValueError("Размеры лабиринта должны быть не менее 3x3.")

    maze = [['█' for _ in range(width)] for _ in range(height)]

    if not start:
        start = (random.randint(1, height - 2), 0)
    if not end:
        end = (random.randint(1, height - 2), width - 1)

    maze[start[0]][start[1]] = '~'
    maze[end[0]][end[1]] = '$'

    for x in range(1, height - 1, 2):
        for y in range(1, width - 1, 2):
            maze[x][y] = ' '

    walls = [(x, y) for x in range(1, height - 1, 2) for y in range(1, width - 1, 2)]
    random.shuffle(walls)

    def is_connected(x, y):
        maze[x][y] = ' '
        visited = set()
        stack = [(x, y)]
        while stack:
            cx, cy = stack.pop()
            if (cx, cy) not in visited:
                visited.add((cx, cy))
                for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                    nx, ny = cx + dx, cy + dy
                    if maze[nx][ny] == ' ':
                        stack.append((nx, ny))
        maze[x][y] = '█'
        return start in visited and end in visited

    for x, y in walls:
        if not is_connected(x, y):
            maze[x][y] = ' '

    return maze

# Пример использования
print("Лабиринт 10x20 со случайными начальной и конечной точками:")
maze = create_maze(10, 20)
for row in maze:
    print(''.join(row))

Лабиринт 10x20 со случайными начальной и конечной точками:
████████████████████
█ █ █ █ █ █ █ █ █ ██
████████████████████
█ █ █ █ █ █ █ █ █ ██
███████████████████$
█ █ █ █ █ █ █ █ █ ██
~███████████████████
█ █ █ █ █ █ █ █ █ ██
████████████████████
████████████████████
